In [9]:
# init
from pyquda.utils import gauge_utils
from pyquda.field import LatticeFermion
from pyquda import core, pyqcu, mpi
import os
import sys
from time import perf_counter
import cupy as cp
test_dir = os.path.dirname(os.path.abspath("./"))
sys.path.insert(0, os.path.join(test_dir, ".."))
os.environ["QUDA_RESOURCE_PATH"] = ".cache"
latt_size = [16, 16, 16, 32]
grid_size = [1, 1, 1, 1]
Lx, Ly, Lz, Lt = latt_size
Nd, Ns, Nc = 4, 4, 3
Gx, Gy, Gz, Gt = grid_size
latt_size = [Lx//Gx, Ly//Gy, Lz//Gz, Lt//Gt]
Lx, Ly, Lz, Lt = latt_size
Vol = Lx * Ly * Lz * Lt
mpi.init(grid_size)
latt_tmp0 = LatticeFermion(latt_size, cp.zeros(
    (Lt, Lz, Ly, Lx, Ns, Nc), cp.complex128))
latt_tmp1 = LatticeFermion(latt_size, cp.zeros(
    (Lt, Lz, Ly, Lx, Ns, Nc), cp.complex128))
latt_shape = (Lt, Lz, Ly, Lx//2, Ns, Nc)
param = pyqcu.QcuParam()
param.lattice_size = latt_size
_dslash = core.getDslash(latt_size, -3.5, 0, 0, anti_periodic_t=False)
kappa = 0.125
U = gauge_utils.gaussGauge(latt_size, 0)
_dslash.loadGauge(U)

Creating Gaussian distrbuted Lie group field with sigma = 1.000000e-01


In [10]:
# give ans first
ans_e = cp.random.random(latt_shape) + 1j * \
    cp.random.random(latt_shape)  # ans_e
ans_o = cp.random.random(latt_shape) + 1j * \
    cp.random.random(latt_shape)  # ans_o

In [11]:
# give x_o, b__o, r, r_tilde, p, v, s, t
x_o = cp.random.random(latt_shape) + 1j * cp.random.random(latt_shape)  # x_o
b_e = cp.zeros(latt_shape, cp.complex128)
b_o = cp.zeros(latt_shape, cp.complex128)
b__o = cp.zeros(latt_shape, cp.complex128)
r = cp.zeros(latt_shape, cp.complex128)
r_tilde = cp.zeros(latt_shape, cp.complex128)
p = cp.zeros(latt_shape, cp.complex128)
s = cp.zeros(latt_shape, cp.complex128)
v = cp.zeros(latt_shape, cp.complex128)
t = cp.zeros(latt_shape, cp.complex128)
# give r_norm2, MAX_ITER, TOL, rho_prev, rho, alpha, omega, beta, kappa
r_norm2 = 0
MAX_ITER = 1e3
TOL = 1e-16
rho_prev = 1
rho = 0
alpha = 1
omega = 1
beta = 0
kappa = 0.125

In [12]:
# define dslash
def dslash_eo(src_o):
    latt_tmp1.data[1, :] = src_o
    pyqcu.dslashQcu(latt_tmp0.even_ptr, latt_tmp1.odd_ptr,
                    U.data_ptr, param, 0)  # D_eo
    return latt_tmp0.data[0, :]


def dslash_oe(src_e):
    latt_tmp1.data[0, :] = src_e
    pyqcu.dslashQcu(latt_tmp0.odd_ptr, latt_tmp1.even_ptr,
                    U.data_ptr, param, 1)  # D_oe
    return latt_tmp0.data[1, :]


def dslash(src_o):
    return src_o-kappa**2*dslash_oe(dslash_eo(src_o))

In [13]:
# give b'_o(b__0)
b__o = (ans_o-kappa*dslash_oe(ans_e))+kappa*dslash_oe((ans_e-kappa*dslash_eo(ans_o)))

In [14]:
def dot(a, b):
    cp.cuda.runtime.deviceSynchronize()
    return cp.inner(a.flatten().conjugate(), b.flatten())


def diff(a, b):
    cp.cuda.runtime.deviceSynchronize()
    return cp.linalg.norm(a - b) / cp.linalg.norm(b)

In [15]:
# bistabcg
r = b__o - dslash(x_o)
r_tilde = r

In [16]:
t0 = perf_counter()
for i in range(1, int(MAX_ITER)):
    # print("## rho:", rho)
    rho = dot(r_tilde, r)
    # print("## beta:", beta)
    beta = (rho / rho_prev) * (alpha / omega)
    p = r + (p - v * omega) * beta
    # v = A * p
    v = dslash(p)
    # print("## alpha:", alpha)
    alpha = rho / dot(r_tilde, v)
    s = r - v * alpha
    # t = A * s
    t = dslash(s)
    # print("## omega:", omega)
    omega = dot(t, s) / dot(t, t)
    x_o = x_o + p * alpha + s * omega
    r = s - t * omega
    r_norm2 = cp.linalg.norm(r)
    print("##{}# r_norm2:{}".format(i, r_norm2))
    # break
    if (r_norm2 < TOL or i == MAX_ITER - 1):
        print("### turns:", i)
        break
    rho_prev = rho
t1 = perf_counter()

##1# r_norm2:52.765551462202296
##2# r_norm2:22.79345910310252


##3# r_norm2:58.042215680551806
##4# r_norm2:21.161017024027608
##5# r_norm2:5.198172843241786
##6# r_norm2:3.532048340604465
##7# r_norm2:2.697426112842914
##8# r_norm2:1.8264028064283278
##9# r_norm2:0.9378145837248625
##10# r_norm2:0.6928953938245896
##11# r_norm2:0.5876740307609957
##12# r_norm2:0.44237495141867383
##13# r_norm2:0.29558247625483836
##14# r_norm2:0.29659199950990134
##15# r_norm2:0.20426239005015848
##16# r_norm2:0.1568724806805465
##17# r_norm2:0.1450322893715671
##18# r_norm2:0.39770013745127597
##19# r_norm2:0.5230259873233651
##20# r_norm2:0.2190927779819697
##21# r_norm2:0.28399291757310585
##22# r_norm2:0.055760789973925105
##23# r_norm2:0.059018532981586616
##24# r_norm2:0.050926519231344405
##25# r_norm2:0.05006405742224656
##26# r_norm2:0.04607056412654988
##27# r_norm2:0.024505936051325294
##28# r_norm2:0.02000462049798889
##29# r_norm2:0.016802352032726082
##30# r_norm2:0.018600214534347875
##31# r_norm2:0.01384136109304529
##32# r_norm2:0.013928079134704

In [17]:
print("### ans_o = ", ans_o[0, 0, 0, 0, :, :])
print("### x_o = ", x_o[0, 0, 0, 0, :, :])
print('## difference: ', diff(x_o, ans_o))
print(f'Qcu time: {t1 - t0} sec')

### ans_o =  [[0.57507219+0.28229903j 0.29311359+0.67457834j 0.76039068+0.30422137j]
 [0.61718769+0.73051242j 0.01002514+0.90243128j 0.00120315+0.39971531j]
 [0.77199059+0.04240324j 0.91965348+0.81740504j 0.36698491+0.18228881j]
 [0.67015843+0.35533373j 0.20936913+0.36436535j 0.47985352+0.7261822j ]]
### x_o =  [[0.57507219+0.28229903j 0.29311359+0.67457834j 0.76039068+0.30422137j]
 [0.61718769+0.73051242j 0.01002514+0.90243128j 0.00120315+0.39971531j]
 [0.77199059+0.04240324j 0.91965348+0.81740504j 0.36698491+0.18228881j]
 [0.67015843+0.35533373j 0.20936913+0.36436535j 0.47985352+0.7261822j ]]
## difference:  8.827126943413841e-16
Quda dslash: 6.165716873000065 sec


In [18]:
x_e = ans_e-kappa*dslash_eo(ans_o)+kappa * \
    dslash_eo(x_o)  # equal to x_o == ans_o
print(diff(x_e, ans_e))

5.027701029605067e-16
